# Домашнее задание «Функции и работа с данными»

### Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [3]:
ratings_df = pd.read_csv('data/ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
def classification(rating):
    if rating <= 2:
        return 'low'
    elif rating <= 4:
        return 'middle'
    else:
        return 'high'


In [6]:
ratings_df['class'] = ratings_df['rating'].apply(classification)
ratings_df.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,middle
1,1,1029,3.0,1260759179,middle
2,1,1061,3.0,1260759182,middle
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,middle


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Результат классификации запишите в отдельный столбец region.



In [11]:
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [12]:
keywords_df = pd.read_csv('data/keywords.csv')
keywords_df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [13]:
def get_region(query):
    for k, v in geo_data.items():
        for city in v:
            if city in query:
                return k
    return 'undefined'


In [16]:
keywords_df['region'] = keywords_df['keyword'].apply(get_region)
# keywords_df.head()
keywords_df[keywords_df.region != 'undefined'].head()


,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [108]:
years = list(range(1950,2011))
movies_df = pd.read_csv('data/movies.csv', usecols=['movieId', 'title'])
ratings_df = pd.read_csv('data/ratings.csv', usecols=['movieId', 'rating'])

In [92]:
# Лектору: Как вариант, в этой задаче можно использовать regex
# import re

# def production_year(title):
#     pattern = r'.\((\d{4}).*\)'
#     result = re.findall(pattern, title)
#     return 1900 if not result else result[0]

# # get_year fn test
# assert production_year('Toy Story (1995)') == '1995'
# assert production_year('Big Bang Theory, The (2007-)') == '2007'
# assert production_year('Fawlty Towers (1975-1979)') == '1975'

In [109]:
def production_year(row):
    for year in years:
        if str(year) in row['title']:
            return year
    return 1900

In [110]:

# add year column
movies_df['year'] = movies_df.apply(production_year, axis=1).astype(int) 
movies_df.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [111]:
ratings_df.head()

,movieId,rating
0,31,2.5
1,1029,3.0
2,1061,3.0
3,1129,2.0
4,1172,4.0


In [112]:
merged_movies_df = ratings_df.merge(movies_df, on='movieId', how='left')
merged_movies_df.head()

,movieId,rating,title,year
0,31,2.5,Dangerous Minds (1995),1995
1,1029,3.0,Dumbo (1941),1900
2,1061,3.0,Sleepers (1996),1996
3,1129,2.0,Escape from New York (1981),1981
4,1172,4.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989),1989


In [120]:
result = merged_movies_df.groupby('year').mean()
result.head(10)

,movieId,rating
year,,
1900,65545.252419,3.602960
1950,3503.427966,3.915254
1951,2605.588477,3.983539
1952,4107.796053,4.000000
1953,4032.321101,3.754587
1954,2867.661850,3.994220
1955,4925.548276,3.751724
1956,5645.423529,3.617647
1957,2750.965190,4.014241
